In [5]:
import numpy as np
import tensorflow as tf
import copy
from sklearn import preprocessing
import datetime
import pickle

In [6]:
class NetAttributes:
    def __init__(self, n_neurons = 100, 
                 learning_rate = 0.003, 
                 num_layers = 1,
                 rnn_type = 2,
                 n_repeats = 2):
        self.n_neurons = n_neurons;
        self.learning_rate = learning_rate;
        self.num_layers = num_layers;
        self.rnn_type = rnn_type;
        self.n_repeats = n_repeats
        self.n_steps = None
        self.n_inputs = None
        self.n_outputs = 1
        
    def set_input_dimension(self, n_steps, n_inputs):
        self.n_steps = n_steps
        self.n_inputs = n_inputs


In [7]:
class NetStates:
    def __init__(self):
        self.prediction_states = None
        self.training_states = None
    

In [8]:
class StatefulLstmModel:
    def __init__(self,
                n_neurons=100,
                learning_rate=0.002,
                num_layers=2,
                rnn_type=1,
                n_repeats=30):

        self.net_attributes = NetAttributes(n_neurons,
                                   learning_rate,
                                   num_layers,
                                   rnn_type,
                                   n_repeats)
        self.net_states = NetStates()
        self.model_initialized = False
    
    def __del__(self):
        if self.sess != None:
            self.sess.close()
    
    def get_batch(self, seq_index, data_train_input, data_train_output):
        X_batch = data_train_input[seq_index:seq_index+1]
        y_batch = data_train_output[seq_index:seq_index+1]
        return X_batch, y_batch
    
    
    def initialize_layers(self):
        layers = None
        net_attributes = self.net_attributes
        if net_attributes.rnn_type == 0:
            layers = [tf.nn.rnn_cell.BasicLSTMCell(net_attributes.n_neurons) 
              for _ in range(net_attributes.num_layers)]
        elif net_attributes.rnn_type == 1:
            layers = [tf.nn.rnn_cell.LSTMCell(net_attributes.n_neurons, use_peepholes=False) 
              for _ in range(net_attributes.num_layers)]
        elif net_attributes.rnn_type == 2:
            layers = [tf.nn.rnn_cell.LSTMCell(net_attributes.n_neurons, use_peepholes=True) 
              for _ in range(net_attributes.num_layers)]
        else:
            print("WRONG")
        return layers
    
    def reset_graph(self, seed=42):
        tf.reset_default_graph()
        tf.set_random_seed(seed)
        np.random.seed(seed)
    
    def create_model(self):
        net_attributes = self.net_attributes
        self.X = tf.placeholder(tf.float32, [None, net_attributes.n_steps, net_attributes.n_inputs])
        self.y = tf.placeholder(tf.float32, [None, net_attributes.n_steps, net_attributes.n_outputs])
        layers = self.initialize_layers()
        cell = tf.nn.rnn_cell.MultiRNNCell(layers)
        self.init_state = tf.placeholder(tf.float32, [net_attributes.num_layers, 2, 1, net_attributes.n_neurons])
        
        state_per_layer_list = tf.unstack(self.init_state, axis=0)
        rnn_tuple_state = tuple(
            [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
             for idx in range(net_attributes.num_layers)]
        )
        
        rnn_outputs, self.new_states = tf.nn.dynamic_rnn(cell, self.X, dtype=tf.float32, 
                                                    initial_state=rnn_tuple_state)
        
        stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, net_attributes.n_neurons])
        stacked_outputs = tf.layers.dense(stacked_rnn_outputs, net_attributes.n_outputs)
        self.outputs = tf.reshape(stacked_outputs, [-1, net_attributes.n_steps, net_attributes.n_outputs])
        
        self.loss = tf.reduce_mean(tf.square(self.outputs - self.y))
        optimizer = tf.train.AdamOptimizer(learning_rate=net_attributes.learning_rate)
        self.training_op = optimizer.minimize(self.loss)

        self.init = tf.global_variables_initializer()
        self.model_initialized = True
    
    # train the model, input is the training data for one cycle
    # input is in the shape: [days, steps, features], the features are 
    # 1. diff, 2. volume. 3. timesteps.
    def fit(self, data_train_input, data_train_output, prediction_period):
        net_attributes = self.net_attributes
        net_states = self.net_states
        n_inputs = data_train_input.shape[2]
        n_steps = data_train_input.shape[1]
        print("net_attributes")
        print(net_attributes)
        net_attributes.set_input_dimension(n_steps, n_inputs)
        batch_size = 1
        days = data_train_input.shape[0]
        
        self.reset_graph()
        self.create_model()
        my_loss_train_list = []
        sess = tf.Session()
        # TODO: load from file.

        self.init.run(session=sess)
        # if this is the first time of fit?
        if self.net_states.training_states == None:
            init_states = np.zeros((net_attributes.num_layers, 2, 1, net_attributes.n_neurons))
        else:
            init_states = self.net_states.training_states
            
        for repeat in range(net_attributes.n_repeats):
            rnn_states = copy.deepcopy(init_states)
            for seq in range(days):
                X_batch, y_batch = self.get_batch(seq, data_train_input, data_train_output)
                feed_dict = {
                        self.X: X_batch,
                        self.y: y_batch,
                        self.init_state: rnn_states}
                my_op, rnn_states, my_loss_train, my_outputs = sess.run([self.training_op, 
                          self.new_states, 
                          self.loss, 
                          self.outputs], feed_dict=feed_dict)

                my_loss_train_list.append(my_loss_train)
                # last repeat , remember the sates
                if seq+1 == prediction_period and repeat == net_attributes.n_repeats-1:
                    # next training loop starts from here
                    training_states = copy.deepcopy(rnn_states)
                my_loss_train_avg = sum(my_loss_train_list) / len(my_loss_train_list)

            print("{} repeat={} training finished, training MSE={}".format(
                datetime.datetime.now().time(),
                repeat, my_loss_train_avg))
        
        self.net_states.training_states = training_states
        self.net_states.prediction_states = rnn_states
        self.sess = sess
        return
    
    def predict_base(self, data_test_input, data_test_output=None):
        net_attributes = self.net_attributes
        net_states = self.net_states
        days = data_test_input.shape[0]
        
        rnn_states = copy.deepcopy(net_states.prediction_states)
        #X, y, init_state, init, training_op, new_states, loss, outputs = self.create_model()
        sess = self.sess
        
        my_loss_test_list = []
        input_shape = data_test_input.shape
        outputs_all_days = np.zeros((input_shape[0], input_shape[1], 1))
        for seq in range(days):
            if data_test_output is None:
                feed_dict = {
                    self.X: data_test_input[seq:seq+1],
                    self.init_state: rnn_states,
                }

                rnn_states, my_outputs = sess.run([self.new_states, self.outputs], feed_dict=feed_dict)
            else:
                feed_dict = {
                    self.X: data_test_input[seq:seq+1],
                    self.y: data_test_output[seq:seq+1],
                    self.init_state: rnn_states,
                }

                rnn_states, my_outputs, my_loss_test = sess.run([self.new_states, 
                                                                 self.outputs, self.loss], feed_dict=feed_dict)
                print("Predicting day:{} testing MSE: {}".format(seq, my_loss_test))
            outputs_all_days[seq] = my_outputs
            
        
        return outputs_all_days
    
    def predict(self, data_test_input):
        return self.predict_base(data_test_input)
        
    
    def predict_and_verify(self, data_test_input, data_test_output):
        return self.predict_base(data_test_input, data_test_output)
    
    
    def get_attributes_filename(self, path):
        return path + '/net_attributes.pkl'
    
    def get_states_filename(self, path, date):
        return path + '/net_states.pkl'
    
    def get_model_filename(self, path, date):
        return path + '/tf_session.ckpt'
    
    def save(self, path, date):
        
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, self.get_model_filename(path))
        with open(self.get_attributes_filename(path), 'wb') as f:
            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(self.net_attributes, f, pickle.HIGHEST_PROTOCOL)
        with open(self.get_states_filename(path), 'wb') as f:
            pickle.dump(self.net_states, f, pickle.HIGHEST_PROTOCOL)
        print("Model saved in path: %s" % path)
        
            
    def load(self, path, date=None):
        # TODO: if date is none, load the latest.
        # restore hyper-params
        with open(self.get_attributes_filename(path), 'rb') as f:
            self.net_attributes = pickle.load(f)

        # restore states
        with open(self.get_states_filename(path), 'rb') as f:
            self.net_states = pickle.load(f)
        
        # 2. restore graph
        if self.model_initialized == False:
            self.reset_graph()
            self.create_model()
        
        # 3. restore session
        saver = tf.train.Saver()
        self.sess = tf.Session()
        saver.restore(self.sess, self.get_model_filename(path))
        print("Model restored.")

In [9]:
class TimeFormat:
    NONE = 0
    DAY = 1
    WEEK = 2

class DataManipulator:
    def __init__(self, beta, ema, time_format, volume_input, use_centralized_bid, 
                split_daily_data, n_training_days):
        self.beta = beta
        self.ema = ema
        self.time_format = time_format
        self.volume_input = volume_input
        self.use_centralized_bid = use_centralized_bid
        self.split_daily_data = split_daily_data
        self.n_training_days = n_training_days
        self.scaler_input = None
        self.scaler_output = None
        
    def volume_transform(self, volume_series):
        return np.log(volume_series+1)
    
    def transform(self, data_all, n_inputs, n_outputs):
        orig_shape = data_all.shape
        data_train_reshape = data_all.reshape((orig_shape[0] * orig_shape[1], orig_shape[2]))
        
        self.scaler_input = preprocessing.MinMaxScaler().fit(data_train_reshape[:,:n_inputs])
        data_train_input_scaled = self.scaler_input.transform(data_train_reshape[:,:n_inputs])
        
        # the invalid step, we change it to zero!
        data_train_input_scaled[~np.any(data_train_reshape, axis=1)] = 0
        data_train_input = data_train_input_scaled.reshape(orig_shape[0], orig_shape[1], n_inputs)
        
        self.scaler_output = preprocessing.MinMaxScaler().fit(data_train_reshape[:,-n_outputs:])
        data_train_output_scaled = self.scaler_output.transform(data_train_reshape[:,-n_outputs:])
        # the invalid step, we change it to zero!
        data_train_output_scaled[~np.any(data_train_reshape, axis=1)] = 0
        data_train_output = data_train_output_scaled.reshape(orig_shape[0], orig_shape[1], n_outputs)
        
        return data_train_input, data_train_output

    def prep_training_data(self, input_path):
        # load numpy file
        npy_file_name = input_path + "/ema{}_beta{}.npy".format(self.ema, self.beta)
        input_np_data = np.load(npy_file_name)
        # check if we have days more than training period
        assert(input_np_data.shape[0] >= self.n_training_days)
        # the diff is the mandatory
        input_columns = [2]
        
        time_format = self.time_format
        
        print("TimeFormat.DAY")
        print(TimeFormat.DAY)
        if time_format == TimeFormat.DAY:
            input_columns += [0]
        elif time_format == TimeFormat.WEEK:
            input_columns += [1]
        
        if self.volume_input == 1:
            input_columns += [3]
        
        output_columns = [4]
        input_np_data = input_np_data[:,:,input_columns + output_columns]
        
        # we must tranform the volume for it is too big.
        if self.volume_input == 1:
            input_np_data[:,:,-2] = self.volume_transform(input_np_data[:,:,-2])
        
        if self.use_centralized_bid == 0:
            # remove all the rows for centralized bid. it should be from 9.01 to 17.24, which is 516-12=504 steps
            input_np_data = input_np_data[:,7:-5,:]
            
        shape = input_np_data.shape
        n_training_sequences = self.n_training_days
        if self.split_daily_data == 1:
            assert(shape[1] % 2 == 0)
            print("split_daily!")
            input_np_data = input_np_data.reshape((shape[0]*2, 
                                                  int(shape[1]/2), 
                                                  shape[2]))
            n_training_sequences *= 2
            
        print("n_inputs:")
        print(len(input_columns))
        return self.transform(input_np_data[:n_training_sequences], len(input_columns), 1)

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import GPy
import GPyOpt

class ValueModel:
    mixed_domain = [{'name': 'n_neurons', 'type': 'discrete', 'domain': tuple(range(20,160,20))},
          {'name': 'learning_rate', 'type': 'discrete', 'domain': (0.001,0.002,0.003,0.004)},
          {'name': 'num_layers', 'type': 'discrete', 'domain': (1,2,3,4)},
          {'name': 'rnn_type', 'type': 'discrete', 'domain': (0,1,2)},
          {'name': 'learning_period', 'type': 'discrete', 'domain': (10,20)},
          {'name': 'prediction_period', 'type': 'discrete', 'domain': (1,2,3,5,6)},
          {'name': 'n_repeats', 'type': 'discrete', 'domain': (1,2,3)},
          {'name': 'beta', 'type': 'discrete', 'domain': (99, 98)},
          {'name': 'ema', 'type': 'discrete', 'domain': (10,20)},
          {'name': 'time_format', 'type': 'discrete', 'domain': (0,1,2)}, #1 for stepofday, 2 for stepofweek
          {'name': 'volume_input', 'type': 'discrete', 'domain': (0,1)},
          {'name': 'use_centralized_bid', 'type': 'discrete', 'domain': (0,1)},
          {'name': 'split_daily_data', 'type': 'discrete', 'domain': (0,1)}
         ]
    
    def __init__(self, stock_name, stock_index, n_training_days):
        self.stock_name = stock_name
        self.stock_index = stock_index
        self.n_training_days = n_training_days
        self.save_path = "model_{}_{}".format(stock_name, n_training_days)
        return
    
    def get_parameter_str(self, X):
        parameter_str = ""
        for i in range(len(self.mixed_domain)):
            parameter_str += self.mixed_domain[i]["name"]
            parameter_str += ':'
            parameter_str += str(X[i])
            parameter_str += ','
        return parameter_str
    
    def get_max_steps(self, groups):
        max_steps = 0
        for index, df in groups:
            df_len = len(df)
            if df_len > max_steps:
                max_steps = df_len
        return max_steps
    
    def prep_data(self, input_csv_path):
        Path(self.save_path).mkdir(exist_ok=True)
        for ema in (10, 20):
            for beta in (99, 98):
                npy_filename = self.save_path + "/ema{}_beta{}.npy".format(ema, beta)
                config = Path(npy_filename)
                if config.is_file():
                    print("file: {} exists, ignore...".format(npy_filename))
                    continue
                    
                input_filename = input_csv_path + "/data-prep-ema{}-beta{}.csv".format(ema, beta)
                if not Path(input_filename).is_file():
                    print("cannot find file: {}, aborting...".format(input_filename))
                    return
                
                print("pre-processing {}".format(input_filename))
                data = pd.read_csv(input_filename, parse_dates=["timestamp"])
                groups = data.set_index('timestamp').groupby(lambda x: x.date())
                n_steps = self.get_max_steps(groups)
                np_data = np.zeros((len(groups), n_steps, 5))
                index = self.stock_index
                column_list = ['step_of_day',
                               'step_of_week',
                               'diff_ema_{}_{}'.format(ema, index), 
                               'volume_{}'.format(index),
                               'value_ema_{}_beta_{}_{}'.format(ema, beta, index)]
                i = 0
                for index, df in groups:
                    np_data[i] = df[column_list]

                    i+=1
                np.save(npy_filename, np_data)
    
    def get_data_prep_desc_filename(self, path):
        return path + '/data_prep_desc.pkl'
    
    def fit(self, input_csv_path, max_iter=300):
        self.prep_data(input_csv_path)
        self.min_result = 0
        print(self.mixed_domain)
        opt_handler = GPyOpt.methods.BayesianOptimization(f=self.opt_func,  # Objective function       
                                     domain=self.mixed_domain,          # Box-constraints of the problem
                                     initial_design_numdata = 20,   # Number data initial design
                                     acquisition_type='EI',        # Expected Improvement
                                     exact_feval = True)           # True evaluations, no sample noise
        opt_handler.run_optimization(max_iter, eps=0)
    
    def opt_func(self, X_list):
        answer = np.zeros((X_list.shape[0], 1))
        for i in range(len(X_list)):
            print(self.get_parameter_str(X_list[i]))
            features = X_list[i]
            answer[i][0], results_list = self.get_result(features)
            #self.draw_step_profit_graph(self.step_profit_list, "step_profit_{}".format(answer[i][0]))
            #self.step_profit_list = []
            if answer[i][0] < self.min_result:
                print("find new opt:{}, {}".format(answer[i][0], self.get_parameter_str(X_list[i])))
                self.min_answer = answer[i][0]
            else:
                print("find result:{}, {}".format(answer[i][0], self.get_parameter_str(X_list[i])))
        return answer

    def sma(self, data, window):
        """
        Calculates Simple Moving Average
        http://fxtrade.oanda.com/learn/forex-indicators/simple-moving-average
        """
        if len(data) < window:
            return None
        return sum(data[-window:]) / float(window)
    
    def ema(self, data, window):
        if len(data) < 2 * window:
            raise ValueError("data is too short")
        c = 2.0 / (window + 1)
        current_ema = self.sma(data[-window*2:-window], window)
        for value in data[-window:]:
            current_ema = (c * value) + ((1 - c) * current_ema)
        return current_ema
    
    
    def get_result(self, features):
        n_neurons = int(features[0])
        learning_rate = features[1]
        num_layers = int(features[2])
        rnn_type = int(features[3])
        learning_period = int(features[4])
        prediction_period = int(features[5])
        n_repeats = int(features[6])
        beta = int(features[7])
        ema = int(features[8])
        time_format = int(features[9])
        volume_input = int(features[10])
        use_centralized_bid = int(features[11])
        split_daily_data = int(features[12])
        
        data_manipulator = DataManipulator(beta, ema, 
                                           time_format, 
                                           volume_input, 
                                           use_centralized_bid, 
                                           split_daily_data, 
                                           self.n_training_days)
        
        data_training_input, data_training_output = data_manipulator.prep_training_data(self.save_path)
        print("data_training_input.shape")
        print(data_training_input.shape)
        print(data_training_output.shape)
        
        # now define the network
        model = StatefulLstmModel(n_neurons, learning_rate, num_layers, rnn_type, n_repeats)
        
        assert(self.n_training_days % prediction_period == 0)
        
        n_training_seq = self.n_training_days
        n_learning_seq = learning_period
        n_prediction_seq = prediction_period
        if split_daily_data == 1:
            n_training_seq *= 2
            n_learning_seq *= 2
            n_prediction_seq *= 2
        
        daily_errors = []
        
        print("start training: training_seq:{}, learning_seq:{}, prediction_seq:{}".format(n_training_seq, 
                                                                                           n_learning_seq, 
                                                                                           n_prediction_seq))
        for i in range(0, n_training_seq-n_prediction_seq-n_learning_seq+1, prediction_period):
            learning_end = i + n_learning_seq
            print("start training from day:{} - day:{}".format(i, learning_end-1))
            model.fit(data_training_input[i:learning_end], data_training_output[:learning_end], n_prediction_seq)
            prediction_end = learning_end + n_prediction_seq
            print("start predicting from day:{} - day:{}".format(learning_end, prediction_end))
            outputs = model.predict_and_verify(data_training_input[learning_end:prediction_end], 
                                     data_training_output[learning_end:prediction_end])
            
            # calculate the error for every day
            y = data_training_output[learning_end:prediction_end]
            # error is a 1-D array for the every day error
            error = np.mean(np.square(outputs-y), axis=(1,2))
            print("The prediction error is: {}".format(error))
            daily_errors += error.tolist()
            
        print("finished")
        print(daily_errors)
        ema = self.ema(daily_errors, int(len(daily_errors)/2))
        print(ema)
       
        return ema, []
    

In [15]:
value_model = ValueModel('Nordea', 5, 30)
value_model.fit('.')

file: model_Nordea_30/ema10_beta99.npy exists, ignore...
file: model_Nordea_30/ema10_beta98.npy exists, ignore...
file: model_Nordea_30/ema20_beta99.npy exists, ignore...
file: model_Nordea_30/ema20_beta98.npy exists, ignore...
[{'name': 'n_neurons', 'type': 'discrete', 'domain': (20, 40, 60, 80, 100, 120, 140)}, {'name': 'learning_rate', 'type': 'discrete', 'domain': (0.001, 0.002, 0.003, 0.004)}, {'name': 'num_layers', 'type': 'discrete', 'domain': (1, 2, 3, 4)}, {'name': 'rnn_type', 'type': 'discrete', 'domain': (0, 1, 2)}, {'name': 'learning_period', 'type': 'discrete', 'domain': (10, 20)}, {'name': 'prediction_period', 'type': 'discrete', 'domain': (1, 2, 3, 5, 6)}, {'name': 'n_repeats', 'type': 'discrete', 'domain': (1, 2, 3)}, {'name': 'beta', 'type': 'discrete', 'domain': (99, 98)}, {'name': 'ema', 'type': 'discrete', 'domain': (10, 20)}, {'name': 'time_format', 'type': 'discrete', 'domain': (0, 1, 2)}, {'name': 'volume_input', 'type': 'discrete', 'domain': (0, 1)}, {'name': 'u

12:39:46.547239 repeat=2 training finished, training MSE=0.014365676915743582
start predicting from day:48 - day:50
Predicting day:0 testing MSE: 0.0012482479214668274
Predicting day:1 testing MSE: 0.0013432762352749705
The prediction error is: [0.00124825 0.00134328]
start training from day:9 - day:48
net_attributes
12:39:50.787112 repeat=0 training finished, training MSE=0.03510788586863782
12:39:54.235473 repeat=1 training finished, training MSE=0.018160416282626102
12:39:57.441149 repeat=2 training finished, training MSE=0.012670128164124132
start predicting from day:49 - day:51
Predicting day:0 testing MSE: 0.0013392241671681404
Predicting day:1 testing MSE: 0.02197883464396
The prediction error is: [0.00133922 0.02197883]
start training from day:10 - day:49
net_attributes
12:40:01.591103 repeat=0 training finished, training MSE=0.039941501469002105
12:40:04.609148 repeat=1 training finished, training MSE=0.020645913285625282
12:40:07.728329 repeat=2 training finished, training MS

12:42:02.140571 repeat=0 training finished, training MSE=0.03434453340232722
12:42:05.743170 repeat=1 training finished, training MSE=0.017923059529493912
12:42:09.083570 repeat=2 training finished, training MSE=0.012457007540312285
start predicting from day:46 - day:58
Predicting day:0 testing MSE: 0.0010918335756286979
Predicting day:1 testing MSE: 0.0010941672371700406
Predicting day:2 testing MSE: 0.0007888751570135355
Predicting day:3 testing MSE: 0.0008057831437326968
Predicting day:4 testing MSE: 0.01960388943552971
Predicting day:5 testing MSE: 0.0062289112247526646
Predicting day:6 testing MSE: 0.0009428272023797035
Predicting day:7 testing MSE: 0.0007605247665196657
Predicting day:8 testing MSE: 0.0003801152342930436
Predicting day:9 testing MSE: 0.0008257267181761563
Predicting day:10 testing MSE: 0.000928998866584152
Predicting day:11 testing MSE: 0.002030081581324339
The prediction error is: [0.00109183 0.00109417 0.00078888 0.00080578 0.01960389 0.00622891
 0.00094283 0.0

12:42:37.402921 repeat=1 training finished, training MSE=0.07129393513023388
12:42:38.320855 repeat=2 training finished, training MSE=0.04940074855306496
start predicting from day:28 - day:32
Predicting day:0 testing MSE: 0.006604355294257402
Predicting day:1 testing MSE: 0.006304363254457712
Predicting day:2 testing MSE: 0.001741852262057364
Predicting day:3 testing MSE: 0.0011977459071204066
The prediction error is: [0.00660436 0.00630436 0.00174185 0.00119775]
start training from day:10 - day:29
net_attributes
12:42:40.073078 repeat=0 training finished, training MSE=0.04666938913869671
12:42:40.937535 repeat=1 training finished, training MSE=0.024560829065740107
12:42:41.809643 repeat=2 training finished, training MSE=0.01708407151066543
start predicting from day:30 - day:34
Predicting day:0 testing MSE: 0.0014694237615913153
Predicting day:1 testing MSE: 0.0009797598468139768
Predicting day:2 testing MSE: 0.0015553911216557026
Predicting day:3 testing MSE: 0.0008262062328867614
The

KeyboardInterrupt: 

In [733]:
# test
model = StatefulLstmModel()

NameError: name 'n_neurons' is not defined

In [673]:
file_name = "np_ema{}_beta{}.npz".format(20, 99)
data_all = np.load(file_name)['arr_0']

In [674]:
def transform(data_all, n_inputs, n_outputs):
    orig_shape = data_all.shape
    data_train_reshape = data_all.reshape((orig_shape[0] * orig_shape[1], orig_shape[2]))

    scaler_input = preprocessing.MinMaxScaler().fit(data_train_reshape[:,:n_inputs])
    data_train_input_scaled = scaler_input.transform(data_train_reshape[:,:n_inputs])

    # the invalid step, we change it to zero!
    data_train_input_scaled[~np.any(data_train_reshape, axis=1)] = 0
    data_train_input = data_train_input_scaled.reshape(orig_shape[0], orig_shape[1], n_inputs)

    scaler_output = preprocessing.MinMaxScaler().fit(data_train_reshape[:,-n_outputs:])
    data_train_output_scaled = scaler_output.transform(data_train_reshape[:,-n_outputs:])
    # the invalid step, we change it to zero!
    data_train_output_scaled[~np.any(data_train_reshape, axis=1)] = 0
    data_train_output = data_train_output_scaled.reshape(orig_shape[0], orig_shape[1], n_outputs)
    return data_train_input, data_train_output, scaler_output

stock_index = 5
input_column_list = [30+stock_index]
output_column_list = [60+stock_index]
all_data = data_all[:,7:-5,input_column_list+output_column_list]
data_train_input, data_train_output, scaler_output = transform(all_data, 1, 1)



print(data_train_input.shape)
print(data_train_output.shape)
print(scaler_output.data_range_)

# TODO: do the scaling outside here!
model.fit(data_train_input[:30,:,:],data_train_output[:30,:,:],2)


(65, 504, 1)
(65, 504, 1)
[0.05327712]
06:07:37.764473 repeat=0 training finished, training MSE=0.05920288419074495
06:07:40.034253 repeat=1 training finished, training MSE=0.030061603643116542


In [675]:
outputs = model.predict(data_train_input[30:32,:,:])
y = data_train_output[30:32,:,:]
print(np.mean(np.square(outputs-y), axis=(1,2)))
model.predict_and_verify(data_train_input[30:32,:,:],y)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.00023405 0.00021665]
sequence:0 test finished, testing MSE=0.00023405496904160827
sequence:1 test finished, testing MSE=0.00021665381791535765


array([[[0.85862219],
        [0.86148429],
        [0.86303729],
        ...,
        [0.84564275],
        [0.84523499],
        [0.84493393]],

       [[0.84995419],
        [0.85443217],
        [0.85830563],
        ...,
        [0.84940988],
        [0.8482694 ],
        [0.84642345]]])

In [676]:
model.save('save')

Model saved in path: save


In [677]:
model.load('save')
model.predict(data_train_input[30:32,:,:])

INFO:tensorflow:Restoring parameters from save/tf_session.ckpt
Model restored.
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


array([[[0.85862219],
        [0.86148429],
        [0.86303729],
        ...,
        [0.84564275],
        [0.84523499],
        [0.84493393]],

       [[0.84995419],
        [0.85443217],
        [0.85830563],
        ...,
        [0.84940988],
        [0.8482694 ],
        [0.84642345]]])

In [678]:
new_m = StatefulLstmModel()
new_m.load('save')

INFO:tensorflow:Restoring parameters from save/tf_session.ckpt
Model restored.


In [679]:
outputs = new_m.predict(data_train_input[30:32,:,:])
y = data_train_output[30:32,:,:]
print(np.mean(np.square(outputs-y), axis=(1,2)))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.00023405 0.00021665]
